<a href="https://colab.research.google.com/github/AnnaVitali/Neuro_Symbolic_AI_example/blob/master/Neuro_SymbolicAI_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install LTNtorch

In [ ]:
import torch
import ltn
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/wine_dataset.csv') # load data
df.drop('quality', axis = 1, inplace = True) # we want a binary classification problem
df['style'] = np.where(df['style'] == 'red', True, False) # change red and withe in true or false
df = df.sample(frac = 1) # shuffle the data taking a sample of the 100%

## Utility Class

In [ ]:
class DataLoader(object):
  # constructor
  def __init__(self, data, labels, batch_size=1, shuffle=True):
    self.data = data
    self.labels = labels
    self.batch_size = batch_size
    self.shuffle = shuffle

  # return the number of batches
  def __len__(self):
    return int (np.ceil(self.data.shape[0] / self.batch_size))

  # describes the logic to run when we iterate over any instance of this class
  def __iter__(self):
    n = self.data.shape(0)
    idx_pos = np.where(self.labels == 1) [0]
    idx_neg = np.where(self.labels == 0) [0]
    np.random.shuffle(idx_pos)
    np.random.shuffle(idx_neg)

    for start_idx in range(0, n, self.batch_size):
      end_idx = min(start_idx + self.batch_size, n)

      # get one positive and one negative sample for each bace, to keep the class balanced
      pos_batch_size = min(self.batch_size // 2, len(idx_pos))
      neg_batch_size = self.batch_size - pos_batch_size
      pos_idx = idx_pos[:pos_batch_size]
      neg_idx = np.random.choice(idx_neg, size = neg_batch_size, replace = False)

      idx = np.concatenate([pos_idx, neg_idx])
      np.random.shuffle(idx)
      data = self.data[idx]
      labels = self.labels[idx]
      yield data, labels

In [ ]:
class ModelA(torch.nn.Module):
  def __init__(self):
    super(ModelA, self).__init__()

    # model definition

    self.sigmoid = torch.nn.Sigmoid()

    self.layer1 = torch.nn.Linear(11, 64)
    self.layer2 = torch.nn.Linear(64, 64)
    self.layer3 = torch.nn.Linear(64, 1)

    self.relu = torch.nn.ReLU()
    self.dropout  = torch.nn.Dropout(p = 0.1)

  def foreward(self, x):
    x = self.relu(self.layer1(x))
    x = self.relu(self.layer2(x))
    x = self.dropout(x)
    return self.sigmoid(self.layer3(x))

## Logic Tensor Network (LTN)

We need to split the sataset into features and labels and then split them into training and testing sets. We standardize our feaures to have a zero mean and unit variance, thus remove scale influence (fo example the siparity between a feature measuring a person's age and another calculating their salary) and to help the NN converge faster

In [ ]:
features = df.drop('style', axis = 1).values
features = (features - features.mean()) / features.std()

In [ ]:
features = torch.tensor(features).to(dtype=torch.float32)
labels = torch.tensor(df['style'].values).to(dtype=torch.float32)

We create the taining and testing dataloader, using a batch-size of 64. we tak the first 91 samples as part of our training set and the remaining for testing. We restrict the training set to such a small amount to higlight the power of NSAI when it comes to small data.

In [ ]:
train_loader = DataLoader(features[:91], labels[:91], 64, True)
test_loader = DataLoader(features[91:], labels[91:], 64, False)

### Defining the kwnoledge base and the Neural Netork Architecture
The next thing to do now, is to extract the kwnoledge base (axioms), and create the Neural Network. we define our predicate, the connecive and the quantifiers.

We define our **predicate** as a simple fedd-forward neural network of three layer:
- input layer (12, 64), translating the dataset's 11 features to 64 neurons
- hidden layer with 64 neurons
- output layer converging to a single neuron

we define a NOT connective and a FOR ALL quantifier. These are the recommended settings by LTNtorch for binary classification.


In [ ]:
A = ltn.Predicate(ModelA())

# create the NOT standard connective
# connective modules contributes to kwnoledge-base extraction by amalgamating aub-formulas with different features
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())

# create the FOR ALL quantifier
# quantifier module determines the formula dimensions for tensor aggregation
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p = 2), quantifier = "f")

### Training the Logic tensor Network

We need a way to evaluate our system, for doing so we consider two aspects:

- the kwnoledge-base satisfaction level (SAT), this metric answer the question of how good the LTN is at learning. We will use this throughout the training process as part of our loss function (maximizing it)
- the classification performance